In [2]:
import numpy as np
import redis
import json
import requests
from bs4 import BeautifulSoup
import httpx
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sentence_transformers import SentenceTransformer
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.nn import functional as F
from collections import defaultdict
from string import punctuation
from nltk.corpus import stopwords
from boilerpy3 import extractors
import spacy
from spacy.lang.en import English
from spacy.lang.ru import Russian
from nltk.corpus import wordnet as wn
from textblob import TextBlob
import tldextract
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk
from urllib.parse import urlparse
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy
import matplotlib.pyplot as plt
import datetime
import joblib
from joblib import load
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

path = '/home/kalacheva/Desktop/'

import pyppeteer
from pyppeteer import launch 
from pyppeteer_stealth import stealth
import nest_asyncio
import asyncio

import sys
sys.path.append(path + 'utils.py')
import utils
from utils import (remove_tld, print_results, predict_url, 
predict_label, check, symbol_ngrams, url_train_data, 
to_train_embeddings, remove_trash, to_token, classify_web,
url_text_classifier_transform, request_text_classifier_transform, ngram_classifier_transform, wait_request)

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from imblearn.over_sampling import RandomOverSampler
from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = SentenceTransformer('all-MPnet-base-v2')

In [3]:
final_categories = ['Internet and Telecom',
'Arts and Entertainment',
'Business and Industry',
'Travel',
'Health',
'Games',
'People and Society',
'Finance',
'Sports',
'News and Media',
'Career and Education',
'Gambling',
'Food and Drink',
'Autos and Vehicles',
'Law and Government',
'Adult',
'Reference',
'Online Technologies',
'Malicious webpages',
'Shopping',
'Advertising',
'Info Security',
'Real Estate',
'Religion',
'Alcohol and Tobacco',
'Hosting and Sharing', 
'Organizations',
'Meaningless Content']

In [5]:
# # Upload train and test datasets:

# train_df = pd.read_csv(path + 'train.csv')
# test_df = pd.read_csv(path + 'test.csv')

In [6]:
# # Change url to get domain ('https://ya.ru/' -> 'ya.ru')

# train_df['url'] = train_df['url'].apply(remove_tld)
# test_df['url'] = test_df['url'].apply(remove_tld)

In [7]:
# # Divide data for train/test columns and labels:

# X_train, y_train = train_df['url'], train_df['category']
# X_test, y_test = test_df['url'], test_df['category']

In [10]:
def category_dataset(category, directory_path, all_categories_df):
    
    print('Collecting category')
    df_category = pd.read_csv(directory_path + f"{category}.txt", header=None, nrows=30000) 
    df_category = df_category[0].str.split(' ', expand=True)
    df_category.loc[:, 'category'] = 1
    
    print('Collecting not_categry')
    df_size = len(df_category)
    df_not_category = all_categories_df.sample(df_size)
    df_not_category = df_not_category.reset_index(drop=True)
    
    df_not_category['category'] = df_not_category['category'].apply(lambda x: 1 if x == f"label_{category}" else 0)    
    
    print('Making one dataset')
    df_not_category.columns = df_category.columns
    df = pd.concat([df_category.reset_index(drop=True), df_not_category.reset_index(drop=True)], axis=0, ignore_index=True)
    
    print('Mixing dataset')
    df = df.sample(frac=1, random_state=42, ignore_index=True)
    
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    print(f"Length of train_df: {len(train_df)}, lenght of test: {len(test_df)}")
    
    return train_df, test_df 

In [16]:
# import numpy as np
# import os
# import pandas as pd
# from tqdm import tqdm

# directory = "/home/kalacheva/Desktop/new/"

# all_categories_df = pd.DataFrame(columns=range(0, 769))

# for label in tqdm(final_categories):  
#     file_path = os.path.join(directory, f"{label}.txt")

#     data = np.loadtxt(file_path, max_rows=750)  
#     label_df = pd.DataFrame(data)
#     label_df["category"] = f"label_{label}"
# #     print(label_df.shape, all_categories_df.shape)
# #     all_categories_df.loc[len(all_categories_df)] = label_df
#     all_categories_df = pd.concat([all_categories_df, pd.DataFrame(label_df)], ignore_index=True)
# all_categories_df = all_categories_df.drop(all_categories_df.columns[768], axis=1)
# all_categories_df.to_csv('/home/kalacheva/Desktop/all_categories_df.csv', index=False)

100%|███████████████████████████████████████████| 28/28 [00:03<00:00,  7.47it/s]


In [18]:
def train_and_save_model(category, save_path, directory_path, all_categories_df):
    
    print(f"Start training for {category} category")
 
    train_df_filtered, test_df_filtered = category_dataset(category, directory_path, all_categories_df)

    X_train, y_train = train_df_filtered.iloc[:, :-1], train_df_filtered['category'].astype('int') 
    X_test, y_test = test_df_filtered.iloc[:, :-1], test_df_filtered['category'].astype('int') 

    classifier = LGBMClassifier()
    classifier.fit(X_train.astype('float'), y_train.astype('int'))
    y_pred = classifier.predict(X_test.astype('float'))
    score = accuracy_score(y_test.astype('int'), y_pred)
    print(f"Score: {score}")
 
    
    joblib.dump(classifier, save_path + category + '.joblib')

In [19]:
# all_categories_df = pd.read_csv(path + 'all_categories_df.csv')

In [20]:
from lightgbm import LGBMClassifier


for i in final_categories:
    train_and_save_model(i, '/home/kalacheva/Desktop/url_models/', '/home/kalacheva/Desktop/url_embeddings/', all_categories_df)

Start training for Internet and Telecom category
Making one dataset
Mixing dataset
Length of train_df: 32000, lenght of test: 8000
Score: 0.852
Start training for Arts and Entertainment category
Making one dataset
Mixing dataset
Length of train_df: 32000, lenght of test: 8000
Score: 0.75025
Start training for Business and Industry category
Making one dataset
Mixing dataset
Length of train_df: 32000, lenght of test: 8000
Score: 0.84775
Start training for Travel category
Making one dataset
Mixing dataset
Length of train_df: 32000, lenght of test: 8000
Score: 0.795
Start training for Health category
Making one dataset
Mixing dataset
Length of train_df: 32000, lenght of test: 8000
Score: 0.824375
Start training for Games category
Making one dataset
Mixing dataset
Length of train_df: 32000, lenght of test: 8000
Score: 0.82625
Start training for People and Society category
Making one dataset
Mixing dataset
Length of train_df: 32000, lenght of test: 8000
Score: 0.840375
Start training for Fin